# In this notebook I will try to connect to a local model directly from LangChain without using LocalApi
Local api is giving me trouble, especially when trying to use GPU.  The performance is much worse than I would expect.  I am going to try to connect directly to the model and see if that helps.

https://python.langchain.com/docs/guides/local_llms

In [2]:
%pip install langchain
%pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir


Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.7 MB 10.6 MB/s eta 0:00:01
     --------- ------------------------------ 1.9/7.7 MB 17.1 MB/s eta 0:00:01
     ------------------- -------------------- 3.8/7.7 MB 26.9 MB/s eta 0:00:01
     ------------------------------- -------- 6.0/7.7 MB 29.7 MB/s eta 0:00:01
     ---------------------------------------- 7.7/7.7 MB 31.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------

In [3]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = LlamaCpp(
    model_path=r"C:\Users\kgage\source\repos\LocalAI\models\luna-ai-llama2-uncensored.Q4_0.gguf",
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [4]:
llm("The first man on the moon was ... Let's think step by step")

# For someone who has never been to space before, the question "What was the first man on the moon?" might seem a bit confusing.➖
Step 1: Ask yourself what you know about outer space. Do you know any facts or information related to space exploration? If so, use that knowledge to narrow down your answer. For example, do you know which country was the first to launch a human into space? 
Step 2: Use your internet search skills and search for "who was the first man on the moon?" or "first lunar landing." This will bring up articles, videos, and other resources that can help you answer the question. 
Step 3: Read through the results and look for information related to the first person to land on the moon. You should be able to find out who he was and when it happened. Some possible answers include Neil Armstrong, Buzz Aldrin, or Michael Collins.

'# For someone who has never been to space before, the question "What was the first man on the moon?" might seem a bit confusing.➖\nStep 1: Ask yourself what you know about outer space. Do you know any facts or information related to space exploration? If so, use that knowledge to narrow down your answer. For example, do you know which country was the first to launch a human into space? \nStep 2: Use your internet search skills and search for "who was the first man on the moon?" or "first lunar landing." This will bring up articles, videos, and other resources that can help you answer the question. \nStep 3: Read through the results and look for information related to the first person to land on the moon. You should be able to find out who he was and when it happened. Some possible answers include Neil Armstrong, Buzz Aldrin, or Michael Collins.'

## Now I'll try to connect to a local database and run a query

In [5]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

server = '127.0.0.1'
database = 'TestDb'
username = 'sa'
password = 'Password123'
driver = 'ODBC Driver 17 for SQL Server'

conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"

# Only include the People table
db = SQLDatabase.from_uri(conn_str)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)

In [6]:
agent_executor.run("You have access to a Microsoft SQL server using the langchain toolkit. In the TestDb database how many records are in the People table?")




> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit
